# Telco Customer Churn for ICP4D

We'll use this notebook to create a machine learning model to predict customer churn.

# 1.0 Install required packages

In [1]:
!pip install psycopg2-binary | tail -n 1
!pip install ibm-ai-openscale==2.1.14 --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1
!pip install --user -U pyspark==2.1.2 --no-cache | tail -n 1
!pip install --upgrade scikit-learn==0.20.3

ERROR: mxnet 1.4.1 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.17.2 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/5e/82/c0de5839d613b82bddd088599ac0bbfbbbcbd8ca470680658352d2c435bd/scikit_learn-0.20.3-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2


# 2.0 Load and Clean Data
We'll load our data as a pandas data frame.

* Highlight the cell below by clicking it.
* Click the `10/01` "Find data" icon in the upper right of the notebook.
* To load the virtualized data created in Exercise-1, choose the `Remote` tab.
* Choose your virtualized data (i.e. User<xyz>.billingProductCustomers), click `Insert to code` and choose `Insert Pandas DataFrame`
* The code to bring the data into the notebook environment and create a Pandas DataFrame will be added to the cell below.
* Run the cell


In [2]:
# Place cursor below and insert the Pandas DataFrame for the Telco churn data
# Make sure the variable is named `df1` for the line `df1 = pd.read_sql(query, con=conn)`

import dsx_core_utils, requests, jaydebeapi, os, io, sys
from pyspark.sql import SparkSession
import pandas as pd
df3 = None
dataSet = dsx_core_utils.get_remote_data_set_info('USER999.billing+products+customers')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
if (sys.version_info >= (3, 0)):
  conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], [dataSource['user'], dataSource['password']])
else:
  conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
query = 'select * from "' + (dataSet['schema'] + '"."' if (len(dataSet['schema'].strip()) != 0) else '') +  dataSet['table'] + '"'

if (dataSet['query']):
    query = dataSet['query']
df3 = pd.read_sql(query, con=conn)
df3.head()




,Churn,Contract,Dependents,MonthlyCharges,PaperlessBilling,Partner,PaymentMethod,SeniorCitizen,TotalCharges,customerID,...,tenure,DeviceProtection,InternetService,MultipleLines,OnlineBackup,OnlineSecurity,PhoneService,StreamingMovies,StreamingTV,TechSupport
0,Yes,Month-to-month,Yes,50.15,Yes,Yes,Electronic check,0,168.15,5896-NPFWW,...,3,Yes,DSL,No,No,No,Yes,No,No,No
1,No,One year,Yes,84.95,Yes,Yes,Bank transfer (automatic),1,4018.05,9978-HYCIN,...,47,No,Fiber optic,No,Yes,No,Yes,No,Yes,No
2,No,Two year,Yes,66.50,Yes,Yes,Credit card (automatic),0,4811.60,8338-QIUNR,...,72,Yes,DSL,Yes,No,Yes,Yes,No,No,Yes
3,No,Two year,Yes,63.30,Yes,Yes,Bank transfer (automatic),0,4189.70,1525-LNLOJ,...,66,Yes,DSL,Yes,No,No,Yes,No,Yes,No
4,No,Month-to-month,No,83.15,No,No,Electronic check,0,2848.45,9450-TRJUU,...,35,No,Fiber optic,Yes,No,No,Yes,No,Yes,No


We'll use the Pandas naming convention `df` for our DataFrame

In [3]:
df = df3

### 2.1 Drop CustomerID feature (column)

In [4]:
df = df.drop('customerID', axis=1)
df.head(5)

,Churn,Contract,Dependents,MonthlyCharges,PaperlessBilling,Partner,PaymentMethod,SeniorCitizen,TotalCharges,gender,tenure,DeviceProtection,InternetService,MultipleLines,OnlineBackup,OnlineSecurity,PhoneService,StreamingMovies,StreamingTV,TechSupport
0,Yes,Month-to-month,Yes,50.15,Yes,Yes,Electronic check,0,168.15,Male,3,Yes,DSL,No,No,No,Yes,No,No,No
1,No,One year,Yes,84.95,Yes,Yes,Bank transfer (automatic),1,4018.05,Male,47,No,Fiber optic,No,Yes,No,Yes,No,Yes,No
2,No,Two year,Yes,66.50,Yes,Yes,Credit card (automatic),0,4811.60,Male,72,Yes,DSL,Yes,No,Yes,Yes,No,No,Yes
3,No,Two year,Yes,63.30,Yes,Yes,Bank transfer (automatic),0,4189.70,Male,66,Yes,DSL,Yes,No,No,Yes,No,Yes,No
4,No,Month-to-month,No,83.15,No,No,Electronic check,0,2848.45,Male,35,No,Fiber optic,Yes,No,No,Yes,No,Yes,No


### 2.2 Examine the data types of the features

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
Churn               7043 non-null object
Contract            7043 non-null object
Dependents          7043 non-null object
MonthlyCharges      7043 non-null float64
PaperlessBilling    7043 non-null object
Partner             7043 non-null object
PaymentMethod       7043 non-null object
SeniorCitizen       7043 non-null int64
TotalCharges        7032 non-null float64
gender              7043 non-null object
tenure              7043 non-null int64
DeviceProtection    7043 non-null object
InternetService     7043 non-null object
MultipleLines       7043 non-null object
OnlineBackup        7043 non-null object
OnlineSecurity      7043 non-null object
PhoneService        7043 non-null object
StreamingMovies     7043 non-null object
StreamingTV         7043 non-null object
TechSupport         7043 non-null object
dtypes: float64(2), int64(2), object(16)
memory usage: 1.1+ MB


### 2.3 Any NaN values should be removed to create a more accurate model. Prior examination shows NaN values for `TotalCharges`

In [6]:
# Check if we have any NaN values
df.isnull().values.any()

True

In [7]:
# Handle missing values for column 8, TotalCharges
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values="NaN", strategy="mean")

df.iloc[:, 8] = imp.fit_transform(df.iloc[:, 8].values.reshape(-1, 1))
df.iloc[:, 8] = pd.Series(df.iloc[:, 8])

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
# Check if we have any NaN values
df.isnull().values.any()

False

# 3.0 Create a model

In [9]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.createDataFrame(df)
df_data.head()

Row(Churn='Yes', Contract='Month-to-month', Dependents='Yes', MonthlyCharges=50.15, PaperlessBilling='Yes', Partner='Yes', PaymentMethod='Electronic check', SeniorCitizen=0, TotalCharges=168.15, gender='Male', tenure=3, DeviceProtection='Yes', InternetService='DSL', MultipleLines='No', OnlineBackup='No', OnlineSecurity='No', PhoneService='Yes', StreamingMovies='No', StreamingTV='No', TechSupport='No')

### 3.1 Split the data into training and test sets

In [12]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)


print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

Number of records for training: 5631
Number of records for evaluation: 1412


### 3.2 Examine the Spark DataFrame Schema
Look at the data types to determine requirements for feature engineering

In [13]:
spark_df.printSchema()

root
 |-- Churn: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- SeniorCitizen: long (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- tenure: long (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- TechSupport: string (nullable = true)



### 3.3 Use StringIndexer to encodes a string column of labels to a column of label indices

In [14]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model


si_gender = StringIndexer(inputCol = 'gender', outputCol = 'gender_IX')
si_Partner = StringIndexer(inputCol = 'Partner', outputCol = 'Partner_IX')
si_Dependents = StringIndexer(inputCol = 'Dependents', outputCol = 'Dependents_IX')
si_PhoneService = StringIndexer(inputCol = 'PhoneService', outputCol = 'PhoneService_IX')
si_MultipleLines = StringIndexer(inputCol = 'MultipleLines', outputCol = 'MultipleLines_IX')
si_InternetService = StringIndexer(inputCol = 'InternetService', outputCol = 'InternetService_IX')
si_OnlineSecurity = StringIndexer(inputCol = 'OnlineSecurity', outputCol = 'OnlineSecurity_IX')
si_OnlineBackup = StringIndexer(inputCol = 'OnlineBackup', outputCol = 'OnlineBackup_IX')
si_DeviceProtection = StringIndexer(inputCol = 'DeviceProtection', outputCol = 'DeviceProtection_IX')
si_TechSupport = StringIndexer(inputCol = 'TechSupport', outputCol = 'TechSupport_IX')
si_StreamingTV = StringIndexer(inputCol = 'StreamingTV', outputCol = 'StreamingTV_IX')
si_StreamingMovies = StringIndexer(inputCol = 'StreamingMovies', outputCol = 'StreamingMovies_IX')
si_Contract = StringIndexer(inputCol = 'Contract', outputCol = 'Contract_IX')
si_PaperlessBilling = StringIndexer(inputCol = 'PaperlessBilling', outputCol = 'PaperlessBilling_IX')
si_PaymentMethod = StringIndexer(inputCol = 'PaymentMethod', outputCol = 'PaymentMethod_IX')


In [15]:
si_Label = StringIndexer(inputCol="Churn", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

### 3.4 Create a single vector

In [16]:
va_features = VectorAssembler(inputCols=['gender_IX',  'SeniorCitizen', 'Partner_IX', 'Dependents_IX', 'PhoneService_IX', 'MultipleLines_IX', 'InternetService_IX', \
                                         'OnlineSecurity_IX', 'OnlineBackup_IX', 'DeviceProtection_IX', 'TechSupport_IX', 'StreamingTV_IX', 'StreamingMovies_IX', \
                                         'Contract_IX', 'PaperlessBilling_IX', 'PaymentMethod_IX', 'TotalCharges', 'MonthlyCharges'], outputCol="features")

### 3.5 Create a pipeline, and fit a model using RandomForestClassifier 
Assemble all the stages into a pipeline. We don't expect a clean linear regression, so we'll use RandomForestClassifier to find the best decision tree for the data.

In [17]:
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_gender, si_Partner, si_Dependents, si_PhoneService, si_MultipleLines, si_InternetService, si_OnlineSecurity, si_OnlineBackup, si_DeviceProtection, \
                            si_TechSupport, si_StreamingTV, si_StreamingMovies, si_Contract, si_PaperlessBilling, si_PaymentMethod, si_Label, va_features, \
                            classifier, label_converter])

model = pipeline.fit(train_data)

In [18]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.703522


# 4.0 Save the model to Cloud Pak for Data

In [19]:
from dsx_ml.ml import save

In [20]:
MODEL_NAME = "telco churn model"

In [21]:
save(name=MODEL_NAME,
    model=model,
    test_data = test_data,
    algorithm_type='Classification',
    description='This is a SparkML Model to Classify Telco Customer Churn Risk')

Using TensorFlow backend.


{'path': '/user-home/999/DSX_Projects/Customer Churn Project/models/churn_model/1',
 'scoring_endpoint': 'https://dsxl-api/v3/project/score/Python36/spark-2.3/Customer%20Churn%20Project/churn_model/1'}

### 4.1 Write the test data without label to a .csv so that we can later use it for batch scoring

In [22]:
write_score_CSV=test_data.toPandas().drop(['Churn'], axis=1)
write_score_CSV.to_csv('../datasets/TelcoCustomerSparkMLBatchScore.csv', sep=',', index=False)

### 4.2 Write the test data to a .csv so that we can later use it for evaluation

In [23]:
write_eval_CSV=test_data.toPandas()
write_eval_CSV.to_csv('../datasets/TelcoCustomerSparkMLEval.csv', sep=',', index=False)

# 5.0 Deploy the model to Watson Machine Learning

### 5.1 Set up Watson Machine Learning client to communicate with Cloud Pak for Data

In [30]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [31]:
wml_credentials = {
  "url": "",
  "instance_id": "icp",
  "username": "",
  "password": ""
}

client = WatsonMachineLearningAPIClient(wml_credentials)
print(client)

## 5.2 List existing deployments and models

In [32]:
client.repository.list_models()
client.deployments.list()

------------------------------------  ---------------------------------------  ------------------------  ---------
GUID                                  NAME                                     CREATED                   FRAMEWORK
------------------------------------  ---------------------------------------  ------------------------  ---------
------------------------------------  ---------------------------------  ------  --------------  ------------------------  ---------  -------------
GUID                                  NAME                               TYPE    STATE           CREATED                   FRAMEWORK  ARTIFACT TYPE
------------------------------------  ---------------------------------  ------  --------------  ------------------------  ---------  -------------


### 5.3 Save model to Watson Machine Learning client

In [35]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM",
               client.repository.ModelMetaNames.NAME: MODEL_NAME}
stored_model = client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=train_data)

### 5.4 Deploy the model to Watson Machine Learning

In [36]:
model_uid = client.repository.get_model_uid(stored_model)
print(model_uid)

e480bd26-aec6-4680-9621-ffd4e6801f96


In [38]:
created_deployment = client.deployments.create(model_uid, name=MODEL_NAME + " deployment")
client.repository.list_models()
client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: 'e480bd26-aec6-4680-9621-ffd4e6801f96' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='361700fe-4f22-4db3-b8af-e85ba34fa6bb'
------------------------------------------------------------------------------------------------


------------------------------------  ---------------------------------------  ------------------------  ---------
GUID                                  NAME                                     CREATED                   FRAMEWORK
e480bd26-aec6-4680-9621-ffd4e6801f96  telco churn model                        2019-09-13T01:14:49.690Z  mllib-2.3
------------------------------------  ------------------------

## Congratulations, you have created a model based on customer churn data, and deployed it to Watson Machine Learning!